In [ ]:
!pip install ultralytics
!pip install boto3

In [ ]:
from ultralytics import YOLO
import boto3
import os
import torch

print(torch.cuda.is_available()) # comment this line if no gpu available

In [ ]:
def download_data_from_s3(client, bucket_name, prefix, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    for obj in client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']:
        if not os.path.exists(os.path.join(os.getcwd(), os.path.dirname(obj['Key']))):
            os.makedirs(os.path.dirname(obj['Key']))
        if obj['Key'][-1] != '/':
            client.download_file(bucket_name, obj['Key'], obj['Key'])

def get_keys(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        if len(lines) != 2:
            raise Exception("Unable to read keys. Invalid file format.")
        return (lines[0][:-1], lines[1])

id, secret = get_keys("aws_keys.txt")
download_data_from_s3(boto3.client('s3', aws_access_key_id=id, aws_secret_access_key=secret), "sagemaker-us-east-1-992382414571", "datasets", "datasets")

In [ ]:
model = YOLO('yolov8m.pt')
model.to('cuda') # comment this line if no gpu available

results = model.train(data="datasets/geoguessr.yaml", 
                      batch=4,
                      epochs=150,  
                      imgsz=1920, 
                      rect=True, 
                      iou=0.7, 
                      visualize=True, 
                      augment=True, 
                      fliplr=0, 
                      mosaic=0, 
                      hsv_h=0, 
                      hsv_s=0, 
                      hsv_v=0, 
                      dropout=0.1)
results = model.val()

In [ ]:
model.export(imgsz=(1088, 1920))